In [20]:
import pandas, scipy, numpy, seaborn
import matplotlib, matplotlib.pyplot as plt

import pybiomart
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [21]:
matplotlib.rcParams.update({'font.size':20, 'xtick.labelsize':20, 'ytick.labelsize':20, 
                            'axes.grid' : True, 'grid.alpha': 0.5, 'grid.linestyle' : ':',
                            'figure.figsize':(8, 5), 'svg.fonttype' : 'none'})

In [22]:
file_directory = "/Users/kja11/OneDrive - Menntaský/PostDoc_Hypothermia/in_silico/Python/"

# II] Data Download

In [23]:
#data all_counts_filtered
df = pandas.read_csv(file_directory+'1) input/RNAseq/transcript_HEK293_temp_all_counts_unfiltered.tsv',
                     sep = '\t')
print(df.shape)
df.head()

(135603, 9)


,gene_id,gene_name,feature_id,s37rep2,s32rep3,s37rep1,s37rep3,s32rep1,s32rep2
0,ENSG00000198034,RPS4X,ENST00000316084,232646,199874,190685,196125,160799,177622
1,ENSG00000142937,RPS8,ENST00000396651,191357,163277,156765,160123,133777,150626
2,ENSG00000182774,RPS17,ENST00000647841,188996,163258,157911,153051,133896,156007
3,ENSG00000229117,RPL41,ENST00000546591,182501,170503,142081,168055,126151,146154
4,ENSG00000142937,RPS8,ENST00000485390,182336,155347,148909,154214,126332,142296


In [24]:
# Import data from hsapiens_gene_ensembl
dataset = pybiomart.Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')
annotation = dataset.query(attributes=['external_gene_name', 'external_transcript_name', 'ensembl_gene_id', 'ensembl_transcript_id',
                                       'gene_biotype', 'transcript_biotype', 'description'])

annotation = annotation.rename(columns = {'Gene stable ID' : 'gene_id',
                                   'Transcript stable ID' : 'transcript_id'})

annotation.set_index('transcript_id', drop=True, inplace=True)
# annotation.set_index('Gene stable ID', inplace=True)
annotation.head(2)

,Gene name,Transcript name,gene_id,Gene type,Transcript type,Gene description
transcript_id,,,,,,
ENST00000387314,MT-TF,MT-TF-201,ENSG00000210049,Mt_tRNA,Mt_tRNA,mitochondrially encoded tRNA-Phe (UUU/C) [Sour...
ENST00000389680,MT-RNR1,MT-RNR1-201,ENSG00000211459,Mt_rRNA,Mt_rRNA,mitochondrially encoded 12S rRNA [Source:HGNC ...


# III] Data transformation

In [25]:
# Organize the df
df = df.rename(columns = {'feature_id' : 'transcript_id'})
df.set_index('transcript_id', drop=True, inplace=True)
df = df.drop(['gene_name', 'gene_id'], axis=1)
print(df.head(1))
# It's not the good column order
df = df[['s37rep1', 's37rep2', 's37rep3', 's32rep1', 's32rep2', 's32rep3']]
df.head()

                 s37rep2  s32rep3  s37rep1  s37rep3  s32rep1  s32rep2
transcript_id                                                        
ENST00000316084   232646   199874   190685   196125   160799   177622


,s37rep1,s37rep2,s37rep3,s32rep1,s32rep2,s32rep3
transcript_id,,,,,,
ENST00000316084,190685,232646,196125,160799,177622,199874
ENST00000396651,156765,191357,160123,133777,150626,163277
ENST00000647841,157911,188996,153051,133896,156007,163258
ENST00000546591,142081,182501,168055,126151,146154,170503
ENST00000485390,148909,182336,154214,126332,142296,155347


In [26]:
# Remove the non expressed transcripts (less than 10 counts)
full_leng = len(df)
df = df[df.max(axis=1) >= 10]

filtr_leng = len(df)
dropped_transcripts = full_leng - filtr_leng

print(f'On {full_leng}, {dropped_transcripts} transcripts do not exceed 10 counts')

df = df.T
df

On 135603, 68143 transcripts do not exceed 10 counts


transcript_id,ENST00000316084,ENST00000396651,ENST00000647841,ENST00000546591,ENST00000485390,ENST00000501597,ENST00000598742,ENST00000600467,ENST00000221975,ENST00000593863,...,ENST00000566034,ENST00000293894,ENST00000651659,ENST00000307940,ENST00000575854,ENST00000648570,ENST00000395360,ENST00000479866,ENST00000698386,ENST00000429967
s37rep1,190685,156765,157911,142081,148909,140774,147451,147063,145697,135375,...,2,2,2,1,1,1,0,0,0,0
s37rep2,232646,191357,188996,182501,182336,180643,178746,178019,176662,163895,...,0,0,0,0,0,0,0,0,0,0
s37rep3,196125,160123,153051,168055,154214,166821,152260,151708,150615,139180,...,7,7,4,0,2,16,24,12,4,1
s32rep1,160799,133777,133896,126151,126332,125378,126652,126273,125238,115231,...,3,3,1,1,11,0,0,0,10,11
s32rep2,177622,150626,156007,146154,142296,145483,138197,137659,136606,127173,...,10,10,13,10,5,1,0,1,4,1
s32rep3,199874,163277,163258,170503,155347,169772,147436,146893,145794,134928,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Preapre the deseq2 metadata
metadata = pandas.DataFrame(zip(df.index, ['37°','37°','37°','32°', '32°', '32°']),
                            columns = ['Sample', 'Condition'])

metadata = metadata.set_index('Sample')
metadata

,Condition
Sample,
s37rep1,37°
s37rep2,37°
s37rep3,37°
s32rep1,32°
s32rep2,32°
s32rep3,32°


In [28]:
%%time
# define the data
dds = DeseqDataSet(counts = df,
                   metadata = metadata,
                   design_factors = "Condition")
print(dds)

#run deseq2
dds.deseq2()

#show results, The name provided in the second element is the level that is used as baseline. 
stat_res = DeseqStats(dds, contrast = ('Condition', '32°', '37°'))

AnnData object with n_obs × n_vars = 6 × 67460
    obs: 'Condition'
    obsm: 'design_matrix'


Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 19.72 seconds.

Fitting dispersion trend curve...
... done in 2.94 seconds.

Fitting MAP dispersions...
... done in 23.03 seconds.

Fitting LFCs...
... done in 14.15 seconds.



CPU times: total: 28.9 s
Wall time: 1min 24s


Refitting 0 outliers.



In [29]:
#save results in a df
stat_res.summary()

res = stat_res.results_df
res

Running Wald tests...
... done in 6.77 seconds.



Log2 fold change & Wald test p-value: Condition 32° vs 37°
                      baseMean  log2FoldChange     lfcSE      stat    pvalue  \
transcript_id                                                                  
ENST00000316084  191445.754149        0.166710  0.237639  0.701529  0.482973   
ENST00000396651  158035.042472        0.184380  0.233656  0.789108  0.430049   
ENST00000647841  157570.432134        0.227071  0.228550  0.993528  0.320453   
ENST00000546591  155707.749717        0.210090  0.256955  0.817615  0.413577   
ENST00000485390  150346.682757        0.171633  0.235535  0.728697  0.466187   
...                        ...             ...       ...       ...       ...   
ENST00000648570       3.044330       -3.844121  2.951246 -1.302542  0.192731   
ENST00000395360       4.117304       -5.327108  4.210418 -1.265221  0.205792   
ENST00000479866       2.225072       -3.362734  3.995841 -0.841558  0.400035   
ENST00000698386       3.133925        1.868918  2.686408  0.6

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
transcript_id,,,,,,
ENST00000316084,191445.754149,0.166710,0.237639,0.701529,0.482973,0.926364
ENST00000396651,158035.042472,0.184380,0.233656,0.789108,0.430049,0.912718
ENST00000647841,157570.432134,0.227071,0.228550,0.993528,0.320453,0.880298
ENST00000546591,155707.749717,0.210090,0.256955,0.817615,0.413577,0.908170
ENST00000485390,150346.682757,0.171633,0.235535,0.728697,0.466187,0.922093
...,...,...,...,...,...,...
ENST00000648570,3.044330,-3.844121,2.951246,-1.302542,0.192731,NaN
ENST00000395360,4.117304,-5.327108,4.210418,-1.265221,0.205792,NaN
ENST00000479866,2.225072,-3.362734,3.995841,-0.841558,0.400035,NaN


In [30]:
# create a df of the result + annotation
df_anno  = pandas.merge(annotation, res, left_index=True, right_index=True, how = 'right')
print(df_anno .shape)
df_anno.head(3)

(67460, 12)


,Gene name,Transcript name,gene_id,Gene type,Transcript type,Gene description,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
transcript_id,,,,,,,,,,,,
ENST00000316084,RPS4X,RPS4X-201,ENSG00000198034,protein_coding,protein_coding,ribosomal protein S4 X-linked [Source:HGNC Sym...,191445.754149,0.166710,0.237639,0.701529,0.482973,0.926364
ENST00000396651,RPS8,RPS8-202,ENSG00000142937,protein_coding,protein_coding,ribosomal protein S8 [Source:HGNC Symbol;Acc:H...,158035.042472,0.184380,0.233656,0.789108,0.430049,0.912718
ENST00000647841,RPS17,RPS17-209,ENSG00000182774,protein_coding,protein_coding,ribosomal protein S17 [Source:HGNC Symbol;Acc:...,157570.432134,0.227071,0.228550,0.993528,0.320453,0.880298


In [31]:
# save to csv
df_anno.to_csv(file_directory+'1) input/RNAseq/from_output/DETranscripts_HEK293_temp_annotated.csv', sep=',')

## Significant ones

In [32]:
# select the significant one (differentially expressed)
df_signif = df_anno[(df_anno.padj < 0.05) & (abs(df_anno.log2FoldChange) > 0.5)]
print(f'{len(df_signif)} transcripts are significantly different between 37° and 32°C')

df_signif.head()

702 transcripts are significantly different between 37° and 32°C


,Gene name,Transcript name,gene_id,Gene type,Transcript type,Gene description,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
transcript_id,,,,,,,,,,,,
ENST00000295470,HNRNPDL,HNRNPDL-201,ENSG00000152795,protein_coding,protein_coding,heterogeneous nuclear ribonucleoprotein D like...,6861.896999,0.906518,0.239715,3.781649,0.000156,0.019366
ENST00000621267,HNRNPDL,HNRNPDL-208,ENSG00000152795,protein_coding,protein_coding,heterogeneous nuclear ribonucleoprotein D like...,6855.061367,0.905946,0.239408,3.784112,0.000154,0.019225
ENST00000602300,HNRNPDL,HNRNPDL-206,ENSG00000152795,protein_coding,protein_coding,heterogeneous nuclear ribonucleoprotein D like...,6406.125721,0.960598,0.239483,4.011131,0.000060,0.009924
ENST00000502762,HNRNPDL,HNRNPDL-203,ENSG00000152795,protein_coding,nonsense_mediated_decay,heterogeneous nuclear ribonucleoprotein D like...,5697.577839,0.808789,0.212906,3.798806,0.000145,0.018611
ENST00000349655,HNRNPDL,HNRNPDL-202,ENSG00000152795,protein_coding,nonsense_mediated_decay,heterogeneous nuclear ribonucleoprotein D like...,5678.544245,0.809719,0.211100,3.835719,0.000125,0.016852


In [33]:
# save to .txt the genes
# all 
numpy.savetxt(file_directory+"3) output/RNAseq/allsignif_transcripts_HEK293.txt", 
              df_signif['Transcript name'].unique(), delimiter="\t", fmt="% s")